### Дипломная работа Александра Соколова

#### Анализ моделей градиентного бустинга LightGBM
Кернел 5 из 5 в разделе ML (отредактирован 21.04.2021)
---

# 1. Импорт библиотек, инициализация глобальных констант
## 1.1. Импорт библиотек

In [ ]:
%load_ext autoreload
%autoreload 2

import pandas as pd
import numpy as np
import tqdm

from sklearn.model_selection import train_test_split, KFold

import lightgbm as lgb

import pickle

np.warnings.filterwarnings('ignore')

In [ ]:
import utils_21042021 as utils

## 1.2. Глобальные константы

In [ ]:
# CURRENT_DIR = './'  # имя текущей директории для локальной машины 
CURRENT_DIR = '../'  # имя текущей директории для каггл

PATH_TO_MODELS = CURRENT_DIR + 'input/models-booster-alfabattle2-sandbox/'

PATH_TO_WORKDIR = CURRENT_DIR + 'working/'

In [ ]:
!pip freeze > requirements.txt

# 2. Вспомогательные функции

In [ ]:
def func_border(a, border):
    """
    функция применяется для создания бинарного вектора из вектора прогнозов по заданной границе
    :border - граница вероятности выше которой относим к дефолтным (1) ниже 0
    """
    if a > border:
        return 1
    else:
        return 0
vfunc = np.vectorize(func_border)

# 3. Импорт данных и моделей
---
## 3.1 Импорт агрегированных данных после препроцессинга
(препроцессинг проводился в [первом кернеле](https://www.kaggle.com/sokolovaleks/sf-dst-10-diplom-1-ml-sokolov))

In [ ]:
%%time
merged_train_data = pd.read_csv('../input/alfabattle2-sandbox/preproc_data_for_boosting/preproc_data_for_boosting/merged_data.csv')
merged_test_data = pd.read_csv('../input/alfabattle2-sandbox/preproc_data_for_boosting/preproc_data_for_boosting/merged_test_data.csv')

## 3.2 Импорт предобученных моделей бустинга (LightGBM и CatBoost) 
(препроцессинг проводился в [втором (LightGBM)](https://www.kaggle.com/sokolovaleks/sf-dst-10-diplom-2-ml-sokolov) и [третьем (CatBoost)](https://www.kaggle.com/sokolovaleks/sf-dst-10-diplom-3-ml-sokolov) кернелах)

In [ ]:
%%time
models_LGBM = []
feats_models = []
subs_models = []
for i_model in range(1,5,1):
    temp_list_models = []
    for i_count_model in range(1,6,1):
        path_to_model = f'model{i_model}/model{i_model}_{i_count_model}.txt'
        model = lgb.Booster(model_file=PATH_TO_MODELS + path_to_model)
        temp_list_models.append(model)
        
    models_LGBM.append(temp_list_models)
    path_to_feats = f'model{i_model}/feats_model{i_model}.pickle'
    with open(PATH_TO_MODELS + path_to_feats, 'rb') as f:
        feats = pickle.load(f)
    feats_models.append(feats)
    path_to_sub = f'model{i_model}/sub_model{i_model}.csv'
    sub = pd.read_csv(PATH_TO_MODELS + path_to_sub)
    subs_models.append(sub)

## 3.3 Проверка импорта моделей

In [ ]:
%%time
for i_models, models in enumerate(models_LGBM):
    score = np.zeros(len(merged_test_data))
    desc_str=f'Прогноз модели номер := {i_models+1} по фолдам:'
    for model in tqdm.tqdm_notebook(models,desc=desc_str):
        score += model.predict(merged_test_data[feats_models[i_models]]) / len(models)
    print(f'Кол-во переменных для модели:= {len(feats_models[i_models])}')
    submission = pd.DataFrame({
        'app_id' : merged_test_data.app_id.values,
        'score': score
    }) 
    submission['score_true']= subs_models[i_models].score
    submission['diff']= submission['score'] - submission['score_true']
    max_ = submission['diff'].max()
    min_ = submission['diff'].min()
    mean_ = submission['diff'].mean()
    print(f'Максимум, минимум и среднее от ошибки прогноза:= {max_:.4f}, {min_:.4f} ,{mean_:.4f}')

# 4. Анализ первой модели (1)
---
## 4.1 Расчет прогноза дефолта 

In [ ]:
targets = merged_train_data.flag

In [ ]:
%%time
features = feats_models[0]
lgb_models = models_LGBM[0]

targets_pred_broba = np.zeros(len(merged_train_data))

for model in tqdm.tqdm_notebook(lgb_models, desc='Прогноз по модели обученной на 5 фолдах:'):
    targets_pred_broba += model.predict(merged_train_data[features])/ len(lgb_models)

## 4.3 Расчет бинарного вектора прогноза (targets_pred) из вероятностного прогноза (pred_proba)

In [ ]:
sum_defolt_train = targets.sum()
sum_all_train = len(targets)

In [ ]:
sort_pred = sorted(targets_pred_broba)
border = sort_pred[sum_all_train-sum_defolt_train]
targets_pred = vfunc(targets_pred_broba, border)

In [ ]:
# в первый раз инициируем глобальную переменную с предыдущим скором нулевыми значениями
utils.last_pred = np.zeros((4,len(targets)))

## 4.4 Метрики, матрицы ошибок и кривые

In [ ]:
utils.test_last_pred(targets, targets_pred, targets_pred_broba,len(features))
df_metrics = utils.all_metrics(targets, targets_pred, targets_pred_broba,len(features))

In [ ]:
df_metrics = df_metrics.rename(columns={'Значение':'model1'})
df_metrics.drop(['Дельта с предыдущим', 'Описание'], axis=1, inplace=True)

In [ ]:
utils.plot_confusion_matrix_double(targets, targets_pred, 1.2, ['Дефолтные', 'Не дефолтные'])

In [ ]:
utils.ROC_and_PR_curves(targets, targets_pred_broba, 1.2)

# 5. Анализ второй модели (2)
---
## 5.1 Расчет прогноза дефолта 

In [ ]:
%%time
features = feats_models[1]
lgb_models = models_LGBM[1]

targets_pred_broba = np.zeros(len(merged_train_data))

for model in tqdm.tqdm_notebook(lgb_models, desc='Прогноз по модели обученной на 5 фолдах:'):
    targets_pred_broba += model.predict(merged_train_data[features])/ len(lgb_models)

## 5.2 Расчет бинарного вектора прогноза (targets_pred) из вероятностного прогноза (pred_proba)

In [ ]:
sort_pred = sorted(targets_pred_broba)
border = sort_pred[sum_all_train-sum_defolt_train]
targets_pred = vfunc(targets_pred_broba, border)

## 5.3 Метрики, матрицы ошибок и кривые

In [ ]:
temp_df = utils.all_metrics(targets, targets_pred, targets_pred_broba,len(features))
df_metrics['model2'] = temp_df['Значение']

In [ ]:
utils.plot_confusion_matrix_double(targets, targets_pred, 1.2, ['Дефолтные', 'Не дефолтные'])

In [ ]:
utils.ROC_and_PR_curves(targets, targets_pred_broba, 1.2)

# 6. Анализ третьей модели (3)
---
## 6.1 Расчет прогноза дефолта 

In [ ]:
%%time
features = feats_models[2]
lgb_models = models_LGBM[2]

targets_pred_broba = np.zeros(len(merged_train_data))

for model in tqdm.tqdm_notebook(lgb_models, desc='Прогноз по модели обученной на 5 фолдах:'):
    targets_pred_broba += model.predict(merged_train_data[features])/ len(lgb_models)

## 6.2 Расчет бинарного вектора прогноза (targets_pred) из вероятностного прогноза (pred_proba)

In [ ]:
sort_pred = sorted(targets_pred_broba)
border = sort_pred[sum_all_train-sum_defolt_train]
targets_pred = vfunc(targets_pred_broba, border)

## 6.3 Метрики, матрицы ошибок и кривые

In [ ]:
temp_df = utils.all_metrics(targets, targets_pred, targets_pred_broba,len(features))
df_metrics['model3'] = temp_df['Значение']

In [ ]:
utils.plot_confusion_matrix_double(targets, targets_pred, 1.2, ['Дефолтные', 'Не дефолтные'])

In [ ]:
utils.ROC_and_PR_curves(targets, targets_pred_broba, 1.2)

# 7. Анализ четвертой модели (4)
---
## 7.1 Расчет прогноза дефолта 

In [ ]:
%%time
features = feats_models[3]
lgb_models = models_LGBM[3]

targets_pred_broba = np.zeros(len(merged_train_data))

for model in tqdm.tqdm_notebook(lgb_models, desc='Прогноз по модели обученной на 5 фолдах:'):
    targets_pred_broba += model.predict(merged_train_data[features])/ len(lgb_models)

## 7.2 Расчет бинарного вектора прогноза (targets_pred) из вероятностного прогноза (pred_proba)

In [ ]:
sort_pred = sorted(targets_pred_broba)
border = sort_pred[sum_all_train-sum_defolt_train]
targets_pred = vfunc(targets_pred_broba, border)

## 7.3 Метрики, матрицы ошибок и кривые

In [ ]:
temp_df = utils.all_metrics(targets, targets_pred, targets_pred_broba,len(features))
df_metrics['model4'] = temp_df['Значение']

In [ ]:
utils.plot_confusion_matrix_double(targets, targets_pred, 1.2, ['Дефолтные', 'Не дефолтные'])

In [ ]:
utils.ROC_and_PR_curves(targets, targets_pred_broba, 1.2)

# 8. Сводная таблица по метрикам моделей

In [ ]:
df_metrics

In [ ]:
df_metrics.to_csv('metrics_models.csv', index=False)